### <span style="color:red">CONFIGURACIÓN: EJECUTAR UNA SOLA VEZ CUANDO SE INICIA EL COLAB.</span>

In [ ]:
!git clone https://github.com/mayala-afi/econometria-mf.git

%cd econometria-mf

!pip install -q -r requirements.txt

## SESIÓN 2 : SERIES TEMPORALES

### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf

from utils import plot_single_line, plot_acf_pacf, predict_plot_multi

import warnings
warnings.filterwarnings('ignore')

### 1. Carga de datos

Vamos a cargar los datos CPIAUCSL desde un Excel descargados de FRED.

In [ ]:
df = pd.read_excel("./data/CPIAUCSL.xlsx", sheet_name="Monthly")

In [ ]:
# Vistazo rápido a los datos
df.head()

### 2. Formateo y limpieza de datos

In [ ]:
# Renombrar columnas
df.rename(columns={"observation_date": "fecha", "CPIAUCSL": "cpi"}, inplace=True)

# Convertir la columna de fechas a tipo datetime (para manipulación temporal)
df["fecha"] = pd.to_datetime(df["fecha"])
df.set_index("fecha", inplace=True)

In [ ]:
# Graficar la serie temporal del CPI
plot_single_line(df[["cpi"]], "cpi", "Índice de Precios al Consumidor (CPIAUCSL)")

### 3. Transformaciones

In [ ]:
df["l_cpi"] = np.log(df["cpi"])
df["infl_m"] = 100 * df["l_cpi"].diff()

df = df.dropna()

In [ ]:
# Graficar la serie temporal de la inflación mensual aproximada (%)
plot_single_line(df[["infl_m"]], "infl_m", "Inflación Mensual Aproximada (%)")

### 4. Prueba ADF - raíz unitaria

In [ ]:
adf_lcpi = adfuller(df["l_cpi"], maxlag=12, regression="ct") # Con tendencia y constante
print(f"ADF Log CPI:\n - Estadístico={adf_lcpi[0]:.4f}\n - p-valor={adf_lcpi[1]:.4f}")

adf_inf = adfuller(df["infl_m"], maxlag=12, regression="c") # Solo con constante
print(f"\nADF Inflación Mensual:\n - Estadístico={adf_inf[0]:.4f}\n - p-valor={adf_inf[1]:.4f}")

### 5. Identificación ACF y PACF

In [ ]:
plot_acf_pacf(df)

### 6. Estimación ARIMA(1,1,1) sobre niveles

In [ ]:
model_111 = ARIMA(df["l_cpi"], order=(1, 1, 1))
results_111 = model_111.fit()
print(results_111.summary())

### 7. Diagnóstico de residuos

In [ ]:
resid = results_111.resid

plot_acf(resid.dropna(), lags=20)

### 8. Predicción sobre inflación (ARIMA 1,0,1)

In [ ]:
model_101 = ARIMA(df["infl_m"], order=(1, 0, 1))
results_101 = model_101.fit()

df["infl_m_est"] = results_101.fittedvalues

_Truncamos la muestra hasta 2023 (out-of-sample)_

In [ ]:
df_train = df[df.index < "2023-12-01"]

In [ ]:
model_oos = ARIMA(df_train["infl_m"], order=(1, 0, 1))
results_oos = model_oos.fit()

df["infl_m_est_out"] = results_oos.predict(start=df.index[0], end=df.index[-1])

In [ ]:
df_plot = df.loc["2018-01-01":, ["infl_m", "infl_m_est", "infl_m_est_out"]]

In [ ]:
predict_plot_multi(
    df_plot,
    df_train,
    title="Predicción ARIMA(1,0,1) sobre inflación",
    labels=["Observado", "Predicho", "Predicho out-of-sample"],
)
